In [1]:
import pandas as pd
import pathlib
import numpy as np
import scipy.sparse as sparse
import re

In [109]:
path_models = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet")
all_models = list(path_models.iterdir())

In [110]:
path_processed = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/all_processed/md_sin_lote_es.parquet")
path_raw = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/all_processed/minors_insiders_outsiders_origen_sin_lot_info.parquet")

In [2]:
path_model = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_without_lote_CPV_15_topics_45_FINAL")
thetas = sparse.load_npz((path_model / "model_data/TMmodel" / "thetas_orig.npz"))
thetas_arr = thetas.toarray()
print(thetas_arr.shape)

(171060, 15)


In [5]:
with (path_model / "model_data/TMmodel" / "tpc_descriptions.txt").open('r', encoding='utf8') as fin:
    topics_keys = [el.strip() for el in fin.readlines()]
topics_keys

['carpintería_exterior, sustitución_ventana, falso_techo, sala_caldera, humedad, colegio_público, sótano, impermeabilización_cubierta, habitación, carpintería_metálico, revestimiento, acuartelamiento_guardia, filtración, enfriadora, aislamiento',
 'carril_bici, repavimentación, reasfaltado, rotonda, rey_juan_carlos, alcantarillado, casco_urbano, ajardinado, maquinaria_autorpropulsado, reordenación, tragsa_adjudicar, paseo_marítimo, glorieta, constitución, barriada',
 'pista_pádel, pista_atletismo, frontón, polideportivo, ciudad_deportiva, pista_tenis, piscina_verano, cubrición_pista, depuración, pista_padel, deportiva, pabellón_polideportivo, piscina_cubierta, estadio, pista_polideportiva',
 'etapa, renovación, pavimentacion, alumbrado_publico, rehabilitacion, canalización, ampliacion, sustitucion, abastecimiento, agua_potable, sondeo, redes, urbanizacion, depósito_regulador, deposito',
 'construcción_nicho, cementerio_municipal, tratamiento_selvícola, drenaje, talud, monte_utilidad, e

In [8]:
#with (path_model / "model_data/TMmodel" / "tpc_labels.txt").open('r', encoding='utf8') as fin:
#    topics_labels = [el.strip() for el in fin.readlines()]
topics_labels = [
  "Reformas y Mantenimiento de Edificios",
  "Infraestructura y Mejoras Urbanas",
  "Construcción y Mantenimiento de Instalaciones Deportivas",
  "Renovación y Mejora de Servicios Públicos",
  "Construcción y Conservación de Espacios Naturales y Cementerios",
  "Rehabilitación y Mejora de Carreteras",
  "Obras de Canalización y Construcción",
  "Desarrollo Regional y Urbano",
  "Infraestructura Ferroviaria y de Alta Velocidad",
  "Energía Renovable y Tratamiento de Aguas Residuales",
  "Eficiencia Energética y Servicios Técnicos",
  "Modernización de Instalaciones Deportivas y Educativas",
  "Energía Renovable y Mejora de Infraestructuras",
  "Arquitectura y Accesibilidad en Edificios",
  "Intervenciones y Mejoras en Infraestructuras Públicas"
]

with open("tpc_labels.txt", 'w') as file:
    for item in topics_labels:
        file.write(item + '\n')

In [114]:
# Find the most representative document for each topic
top_docs_per_topic = []

for theta_k in thetas_arr.T:  
    sorted_docs_indices = np.argsort(theta_k)[::-1]  # # Sort the documents based on their proportion for the current topic in descending order
    top_10 = sorted_docs_indices[:10]
    top_docs_per_topic.append(top_10)
    
top_docs_per_topic

[array([ 777963, 2010218, 2137151,  455745,  542262, 1925851, 1014249,
         149000,  903088,  149014]),
 array([  74910, 2051642, 1348314,  229705,  429658,  229693,  429673,
           2650, 2007815, 2051655]),
 array([1715657,  729196, 2339669, 2339736, 2339866, 1992437, 2642522,
        1992301,  329138, 2340037]),
 array([2047735, 2240130, 1156191,  871858, 2488185,  457997,  794336,
        2488282, 1254895,  457924]),
 array([2296987, 2592787, 2148968, 1793863, 1793872, 2593243, 2593242,
        1477174, 1477022, 1680181]),
 array([2161943,  705893, 2170807,  198382, 1747062, 2394561, 1025069,
        2641758, 1270183, 2131874]),
 array([2438430, 2464198,  363852, 1907069, 2178886,  118295, 2325873,
        1424704, 2155081, 2606658]),
 array([2471664,  101423, 2382863, 2112075, 1988324, 2633489, 2470634,
        2603579, 2382878, 2305361]),
 array([1131745,  861161, 1035263, 1059155, 1594319, 1594317,  378800,
        1663635, 1594310, 1059193]),
 array([1739845, 1140700, 12

In [115]:
df_processed = pd.read_parquet(path_processed)
df_raw = pd.read_parquet(path_raw)
place_with_lote = pd.merge(df_processed,df_raw,how='left', on='id_tm')
place_with_lote

,id_tm,raw_text,lemmas,lang,origen,title,summary,text
0,0,Reforma de elementos de ventilación exterior d...,reforma elemento ventilación exterior aire_aco...,es,minors,Reforma de elementos de ventilación exterior d...,Id licitación: 000103/2017-1069; Órgano de Con...,Reforma de elementos de ventilación exterior d...
1,1,Servicios de calibrado y certificado de dos de...,servicios calibrado certificado detector portá...,es,minors,Servicios de calibrado y certificado de dos de...,Id licitación: 29-2017-II; Órgano de Contratac...,Servicios de calibrado y certificado de dos de...
2,2,"Patrocinio menor proyecto "" Activitats C.I.N.E...",patrocinio activitats id contratación agencia ...,es,minors,"Patrocinio menor proyecto "" Activitats C.I.N.E""",Id licitación: 013-07-2018; Órgano de Contrata...,"Patrocinio menor proyecto "" Activitats C.I.N.E..."
3,3,Redacción EPIA - Legalización antena emisora d...,epio legalización antena_emisor radio narcea i...,es,minors,Redacción EPIA - Legalización antena emisora d...,Id licitación: CON/2017/51; Órgano de Contrata...,Redacción EPIA - Legalización antena emisora d...
4,4,Obras de reparación del Centro de Información ...,centro información naturaleza rellano id contr...,es,minors,Obras de reparación del Centro de Información ...,Id licitación: 000047/2017-1069; Órgano de Con...,Obras de reparación del Centro de Información ...
...,...,...,...,...,...,...,...,...
2944100,3110255,Contrato de servicios para la prestación del s...,contrato prestación socorrismo_piscina id cont...,es,outsiders,Contrato de servicios para la prestación del s...,Id licitación: 3092_100180P; Órgano de Contrat...,Contrato de servicios para la prestación del s...
2944101,3110256,concesion de servicios de peluquerías en 10 cp...,concesion cpa dependiente dt id contr delegaci...,es,outsiders,concesion de servicios de peluquerías en 10 cp...,Id licitación: CONTR 2023 0000922771; Órgano d...,concesion de servicios de peluquerías en 10 cp...
2944102,3110258,Contrato de concesión de servicios para la ges...,contrato concesión integral cfa id contr_agenc...,es,outsiders,Contrato de concesión de servicios para la ges...,Id licitación: CONTR 2023 0001215494; Órgano d...,Contrato de concesión de servicios para la ges...
2944103,3110259,"Investigación, validación, verificación y gest...",investigación validación verificación document...,es,outsiders,"Investigación, validación, verificación y gest...",Id licitación: CONTR 2023 0001095699; Órgano d...,"Investigación, validación, verificación y gest..."


In [125]:
top_docs_dict = {}

for topic_idx, top_docs_indices in enumerate(top_docs_per_topic):
    # Extract the IDs of the top documents for the current topic
    top_docs_ids = place_with_lote.loc[top_docs_indices, 'id_tm'].tolist()
    
    # Store the top document IDs in the dictionary
    top_docs_dict[topic_idx] = top_docs_ids

# Print the top documents for each topic
for topic, top_docs_ids in top_docs_dict.items():
    print(f"{'='*int(len(topics_keys[topic])*0.1)}")
    print(f"Topic {topic}: {topics_labels[topic]}:\n\t {' '.join(topics_keys[topic].split()[0:10])[:-1]}")
    print(f"{'='*int(len(topics_keys[topic])*0.1)}")
    for doc_id in top_docs_ids[0:2]:
        print(f"{'-'*int(len(topics_keys[topic])*0.1)}")
        print(f"- Document ID: {doc_id}\n")
        print(f"\t TITLE: {place_with_lote[place_with_lote.id_tm == doc_id].title.values.tolist()[0]}")
        print(f"\t SUMMARY: {place_with_lote[place_with_lote.id_tm == doc_id].summary.values.tolist()[0]}")

Topic 0: :
	 seguridad_social, medicamento_extranjero, centro_penitenciario, ciudad_deportiva, acondicionado, aire_acondicionado, cerradura, autofacturae_gr, modelo_fotocopiadora, extremeño_promoción
------------------------------------------------------------------------------------------------------------------------------------------
- Document ID: 783419

	 TITLE: Servicios de pequeñas reparaciónes de electricidad, consumibles y/o sustitución bombillas
	 SUMMARY: Id licitación: CTRO/37/00/2021/0004; Órgano de Contratación: Director Gerente de FREMAP; Importe: 109.82 EUR; Estado: RES
------------------------------------------------------------------------------------------------------------------------------------------
- Document ID: 2032440

	 TITLE: vigilancia ambiental y arqueológica de la ejecución del proyecto de recuperación de la encañizada del ventorillo
	 SUMMARY: Id licitación: 2022/054595/1; Órgano de Contratación: Consejería de Agua, Agricultura, Ganadería y Pesca de la